## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,71.9667,102.5000,17.46,85,98,12.75,overcast clouds
1,1,Yar-Sale,RU,66.8333,70.8333,30.96,98,100,15.77,light snow
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,-9.40,70,1,10.36,clear sky
3,3,Flinders,AU,-34.5833,150.8552,50.00,50,6,12.66,clear sky
4,4,Chapais,CA,49.7834,-74.8492,59.00,29,75,12.66,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you want for your trip? 75
What is the maximum temperature you want for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Bathsheba,BB,13.2167,-59.5167,80.60,65,20,13.80,volcanic ash
13,13,Rockport,US,28.0206,-97.0544,82.99,54,1,11.50,clear sky
20,20,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,84.16,60,100,2.89,overcast clouds
22,22,Sur,OM,22.5667,59.5289,81.99,62,0,9.66,clear sky
24,24,Cabedelo,BR,-6.9811,-34.8339,78.80,100,40,8.05,scattered clouds
31,31,Tautira,PF,-17.7333,-149.1500,86.00,66,75,9.22,broken clouds
33,33,Rikitea,PF,-23.1203,-134.9692,78.21,75,7,10.07,light rain
39,39,Namibe,AO,-15.1961,12.1522,80.47,69,53,5.41,broken clouds
42,42,Georgetown,MY,5.4112,100.3354,82.00,89,20,3.44,few clouds
43,43,Puerto Ayora,EC,-0.7393,-90.3518,82.40,74,40,9.22,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Bathsheba,BB,80.60,volcanic ash,13.2167,-59.5167,
13,Rockport,US,82.99,clear sky,28.0206,-97.0544,
20,Sao Miguel Do Araguaia,BR,84.16,overcast clouds,-13.2750,-50.1628,
22,Sur,OM,81.99,clear sky,22.5667,59.5289,
24,Cabedelo,BR,78.80,scattered clouds,-6.9811,-34.8339,
31,Tautira,PF,86.00,broken clouds,-17.7333,-149.1500,
33,Rikitea,PF,78.21,light rain,-23.1203,-134.9692,
39,Namibe,AO,80.47,broken clouds,-15.1961,12.1522,
42,Georgetown,MY,82.00,few clouds,5.4112,100.3354,
43,Puerto Ayora,EC,82.40,scattered clouds,-0.7393,-90.3518,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel " + hotels["results"][0]["name"] + " found.")
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel Atlantis Hotel found.
Hotel Holiday Inn Express & Suites Rockport - Bay View found.
Hotel Hotel Executivo Palace found.
Hotel Sur Plaza Hotel found.
Hotel Hotel Pigalle found.
Hotel Pension Oaoa found.
Hotel Pension Maro'i found.
Hotel Hotel Chik Chik Namibe found.
Hotel Cititel Penang found.
Hotel Finch Bay Galapagos Hotel found.
Hotel Requinte Hotel found.
Hotel Casa Tia Villa found.
Hotel Máncora Marina Hotel found.
Hotel Mercure Padang found.
Hotel Hotel Real found.
Hotel Keleti Beach Resort found.
Hotel Hotel Quinta Antigua found.
Hotel Villa Enata found.
Hotel Best Western Plus Abbey Inn found.
Hotel Hotel Tesoro Los Cabos found.
Hotel Niyama Private Islands Maldives found.
Hotel Iglesia morava renovada found.
Hotel Marriott's Ko Olina Beach Club found.
Hotel Dhidhoo beach found.
Hotel Hospitality Carnarvon found.
Hotel Shanti Maurice Resort & Spa found.
Hotel Isles Sunset Lodge found.
Hotel Shandrani Beachcomber Resort & Spa found.
Hotel not found... skipping.
Hotel Scoop 

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Bathsheba,BB,80.60,volcanic ash,13.2167,-59.5167,Atlantis Hotel
13,Rockport,US,82.99,clear sky,28.0206,-97.0544,Holiday Inn Express & Suites Rockport - Bay View
20,Sao Miguel Do Araguaia,BR,84.16,overcast clouds,-13.2750,-50.1628,Hotel Executivo Palace
22,Sur,OM,81.99,clear sky,22.5667,59.5289,Sur Plaza Hotel
24,Cabedelo,BR,78.80,scattered clouds,-6.9811,-34.8339,Hotel Pigalle


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))